# Hyperparameter Tuning e Otimização

## Objetivos

- Compreender a diferença entre parâmetros e hiperparâmetros
- Implementar Grid Search e Random Search
- Aplicar otimização bayesiana
- Evitar overfitting na seleção de hiperparâmetros

## Pré-requisitos

- Cross validation
- Conceitos de overfitting
- Algoritmos de ML básicos


## 1. Parâmetros vs Hiperparâmetros

- **Parâmetros**: Aprendidos durante o treinamento (pesos, coeficientes)
- **Hiperparâmetros**: Definidos antes do treinamento (learning rate, C, max_depth)

Exemplos por algoritmo:

- **Linear Regression**: Parâmetros = coeficientes; Hiperparâmetros = regularização
- **Random Forest**: Parâmetros = splits das árvores; Hiperparâmetros = n_estimators, max_depth
- **SVM**: Parâmetros = support vectors; Hiperparâmetros = C, gamma, kernel


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.datasets import make_classification, load_breast_cancer
from sklearn.model_selection import (
    GridSearchCV,
    RandomizedSearchCV,
    cross_val_score,
    train_test_split,
    validation_curve,
    learning_curve,
)
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report
import warnings

warnings.filterwarnings("ignore")

# Carregar dados
data = load_breast_cancer()
X, y = data.data, data.target
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print(f"Shape treino: {X_train.shape}")
print(f"Shape teste: {X_test.shape}")
print(f"Classes: {np.unique(y)}")

## 2. Baseline: Modelo com Hiperparâmetros Padrão


In [ ]:
# Testar modelos com hiperparâmetros padrão
models_baseline = {
    "Random Forest": RandomForestClassifier(random_state=42),
    "SVM": SVC(random_state=42),
    "Logistic Regression": LogisticRegression(random_state=42, max_iter=1000),
}

print("=== Performance com Hiperparâmetros Padrão ===")
baseline_scores = {}
for name, model in models_baseline.items():
    scores = cross_val_score(model, X_train, y_train, cv=5, scoring="accuracy")
    baseline_scores[name] = scores
    print(f"{name:20}: {scores.mean():.3f} ± {scores.std():.3f}")

## 3. Grid Search - Busca Exaustiva


In [ ]:
# Grid Search para Random Forest
print("=== Grid Search: Random Forest ===")

rf_param_grid = {
    "n_estimators": [50, 100, 200],
    "max_depth": [3, 5, 7, None],
    "min_samples_split": [2, 5, 10],
    "min_samples_leaf": [1, 2, 4],
}

rf = RandomForestClassifier(random_state=42)
rf_grid = GridSearchCV(
    rf, rf_param_grid, cv=5, scoring="accuracy", n_jobs=-1, verbose=1
)

rf_grid.fit(X_train, y_train)

print(f"Melhores parâmetros: {rf_grid.best_params_}")
print(f"Melhor score CV: {rf_grid.best_score_:.3f}")
print(f"Score no teste: {rf_grid.score(X_test, y_test):.3f}")

In [ ]:
# Visualizar resultados do Grid Search
results_df = pd.DataFrame(rf_grid.cv_results_)

# Top 10 melhores combinações
print("\n=== Top 10 Melhores Combinações ===")
top_results = results_df.nlargest(10, "mean_test_score")
for idx, row in top_results.iterrows():
    print(
        f"Score: {row['mean_test_score']:.3f} ± {row['std_test_score']:.3f} | "
        f"Params: {row['params']}"
    )

# Heatmap para n_estimators vs max_depth
pivot_table = results_df.pivot_table(
    values="mean_test_score",
    index="param_max_depth",
    columns="param_n_estimators",
    aggfunc="mean",
)

plt.figure(figsize=(10, 6))
import seaborn as sns

sns.heatmap(pivot_table, annot=True, fmt=".3f", cmap="viridis")
plt.title("Grid Search: n_estimators vs max_depth")
plt.xlabel("n_estimators")
plt.ylabel("max_depth")
plt.show()

## 4. Random Search - Busca Aleatória


In [ ]:
from scipy.stats import randint, uniform

print("=== Random Search: Random Forest ===")

# Distribuições para Random Search
rf_param_dist = {
    "n_estimators": randint(50, 300),
    "max_depth": [3, 5, 7, 10, None],
    "min_samples_split": randint(2, 20),
    "min_samples_leaf": randint(1, 10),
    "max_features": ["sqrt", "log2", None],
    "bootstrap": [True, False],
}

rf_random = RandomizedSearchCV(
    RandomForestClassifier(random_state=42),
    rf_param_dist,
    n_iter=100,  # Número de combinações a testar
    cv=5,
    scoring="accuracy",
    n_jobs=-1,
    random_state=42,
    verbose=1,
)

rf_random.fit(X_train, y_train)

print(f"Melhores parâmetros: {rf_random.best_params_}")
print(f"Melhor score CV: {rf_random.best_score_:.3f}")
print(f"Score no teste: {rf_random.score(X_test, y_test):.3f}")

# Comparar tempos de execução
import time

# Grid Search limitado
start_time = time.time()
small_grid = GridSearchCV(
    RandomForestClassifier(random_state=42),
    {"n_estimators": [50, 100], "max_depth": [3, 5]},
    cv=3,
)
small_grid.fit(X_train, y_train)
grid_time = time.time() - start_time

# Random Search
start_time = time.time()
quick_random = RandomizedSearchCV(
    RandomForestClassifier(random_state=42),
    rf_param_dist,
    n_iter=10,
    cv=3,
    random_state=42,
)
quick_random.fit(X_train, y_train)
random_time = time.time() - start_time

print(f"\nTempo Grid Search (4 combinações): {grid_time:.2f}s")
print(f"Tempo Random Search (10 combinações): {random_time:.2f}s")

## 5. Validation Curve - Analisando Um Hiperparâmetro


In [ ]:
# Validation curve para SVM - parâmetro C
C_values = np.logspace(-3, 2, 20)  # 0.001 a 100

train_scores, val_scores = validation_curve(
    SVC(random_state=42),
    X_train,
    y_train,
    param_name="C",
    param_range=C_values,
    cv=5,
    scoring="accuracy",
    n_jobs=-1,
)

# Calcular médias e desvios
train_mean = train_scores.mean(axis=1)
train_std = train_scores.std(axis=1)
val_mean = val_scores.mean(axis=1)
val_std = val_scores.std(axis=1)

# Plotar validation curve
plt.figure(figsize=(10, 6))
plt.semilogx(C_values, train_mean, "o-", color="blue", label="Treino")
plt.fill_between(
    C_values, train_mean - train_std, train_mean + train_std, alpha=0.2, color="blue"
)

plt.semilogx(C_values, val_mean, "o-", color="red", label="Validação")
plt.fill_between(
    C_values, val_mean - val_std, val_mean + val_std, alpha=0.2, color="red"
)

plt.xlabel("Parâmetro C")
plt.ylabel("Accuracy")
plt.title("Validation Curve: SVM - Parâmetro C")
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

# Encontrar melhor C
best_c_idx = np.argmax(val_mean)
best_c = C_values[best_c_idx]
print(f"Melhor C: {best_c:.3f} (Accuracy: {val_mean[best_c_idx]:.3f})")

## 6. Pipeline com Hyperparameter Tuning


In [ ]:
# Pipeline com pré-processamento + modelo
print("=== Pipeline com Grid Search ===")

# Criar pipeline
pipe = Pipeline([("scaler", StandardScaler()), ("classifier", SVC(random_state=42))])

# Parâmetros do pipeline
pipe_param_grid = {
    "classifier__C": [0.1, 1, 10, 100],
    "classifier__gamma": ["scale", "auto", 0.01, 0.1, 1],
    "classifier__kernel": ["rbf", "linear", "poly"],
}

pipe_grid = GridSearchCV(pipe, pipe_param_grid, cv=5, scoring="accuracy", n_jobs=-1)

pipe_grid.fit(X_train, y_train)

print(f"Melhores parâmetros: {pipe_grid.best_params_}")
print(f"Melhor score CV: {pipe_grid.best_score_:.3f}")
print(f"Score no teste: {pipe_grid.score(X_test, y_test):.3f}")

# Relatório detalhado
y_pred = pipe_grid.predict(X_test)
print("\n=== Relatório de Classificação ===")
print(classification_report(y_test, y_pred, target_names=data.target_names))

## 7. Evitando Overfitting na Seleção de Hiperparâmetros


In [ ]:
# Demonstrar overfitting com muitas tentativas
def simulate_hyperparameter_overfitting(n_trials=100):
    # Simular busca com muitas tentativas aleatórias
    best_scores = []

    for trial in range(1, n_trials + 1):
        # Gerar parâmetros aleatórios
        random_params = {
            "n_estimators": np.random.randint(50, 200),
            "max_depth": np.random.choice([3, 5, 7, None]),
            "min_samples_split": np.random.randint(2, 10),
        }

        # Avaliar com CV
        model = RandomForestClassifier(random_state=42, **random_params)
        scores = cross_val_score(model, X_train, y_train, cv=3)

        # Manter o melhor até agora
        if trial == 1:
            best_score = scores.mean()
        else:
            best_score = max(best_score, scores.mean())

        best_scores.append(best_score)

    return best_scores


# Simular overfitting
overfitting_scores = simulate_hyperparameter_overfitting(100)

plt.figure(figsize=(10, 6))
plt.plot(range(1, 101), overfitting_scores, "b-", alpha=0.7)
plt.xlabel("Número de Tentativas")
plt.ylabel("Melhor Score CV até o momento")
plt.title("Overfitting na Seleção de Hiperparâmetros")
plt.grid(True, alpha=0.3)
plt.show()

print(f"Score inicial: {overfitting_scores[0]:.3f}")
print(f"Score após 100 tentativas: {overfitting_scores[-1]:.3f}")
print(f"Melhoria aparente: {overfitting_scores[-1] - overfitting_scores[0]:.3f}")

## 8. Comparação Final de Todas as Abordagens


In [ ]:
# Resumo comparativo
print("=== Resumo Comparativo ===")

models_final = {
    "Baseline RF": RandomForestClassifier(random_state=42),
    "Grid Search RF": rf_grid.best_estimator_,
    "Random Search RF": rf_random.best_estimator_,
    "Pipeline SVM": pipe_grid.best_estimator_,
}

final_results = {}
for name, model in models_final.items():
    # Score de validação cruzada
    cv_scores = cross_val_score(model, X_train, y_train, cv=5)

    # Score no conjunto de teste
    model.fit(X_train, y_train)
    test_score = model.score(X_test, y_test)

    final_results[name] = {
        "CV_mean": cv_scores.mean(),
        "CV_std": cv_scores.std(),
        "Test_score": test_score,
    }

    print(
        f"{name:20}: CV = {cv_scores.mean():.3f} ± {cv_scores.std():.3f}, "
        f"Test = {test_score:.3f}"
    )

# Visualizar resultados finais
models = list(final_results.keys())
cv_means = [final_results[m]["CV_mean"] for m in models]
cv_stds = [final_results[m]["CV_std"] for m in models]
test_scores = [final_results[m]["Test_score"] for m in models]

x = np.arange(len(models))
width = 0.35

plt.figure(figsize=(12, 6))
plt.bar(
    x - width / 2,
    cv_means,
    width,
    yerr=cv_stds,
    label="Cross Validation",
    alpha=0.7,
    capsize=5,
)
plt.bar(x + width / 2, test_scores, width, label="Test Set", alpha=0.7)

plt.xlabel("Modelos")
plt.ylabel("Accuracy")
plt.title("Comparação Final: CV vs Test Set")
plt.xticks(x, models, rotation=45)
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## 9. Resumo e Boas Práticas

### Estratégias de Busca:

1. **Grid Search**: Busca exaustiva, boa para poucos parâmetros
2. **Random Search**: Mais eficiente para muitos parâmetros
3. **Otimização Bayesiana**: Para casos complexos e caros

### Boas Práticas:

- Usar **nested cross-validation** para estimativa não-viesada
- Sempre usar **pipeline** para evitar data leakage
- Definir **budget de tempo/tentativas** para evitar overfitting
- Começar com **ranges amplos**, depois refinar
- **Validar no conjunto de teste** apenas uma vez no final

### Cuidados:

- Muitas tentativas podem levar a overfitting nos hiperparâmetros
- Always use proper cross-validation
- Considerar custo computacional vs ganho de performance
- Documentar e reproduzir configurações
